In [7]:
import pandas as pd
from pathlib import Path
from time import sleep
import re

import requests
from bs4 import BeautifulSoup as bs
import json
from zipfile import ZipFile

from datetime import datetime, timedelta
from dateutil.rrule import rrule, WEEKLY, DAILY

from selenium import webdriver
from selenium.webdriver import Keys, ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

In [2]:
wd = Path('projects/deputados-nao-declaram-gastos-com-google-e-facebook')

ad_library_main_page = Path(wd / 'ad-library-main-page')
single_ad_pages = Path(wd / 'single-ad-pages')
downloads = Path(wd / 'downloads')
resultados = Path(wd / 'resultados')

for folder in [single_ads_pages, ad_library_main_page, downloads, resultados]:
    if not folder.exists():
        folder.mkdir(parents=True)

# Gastos da Câmara

In [6]:
exps_URL = 'https://www.camara.leg.br/cotas/Ano-2023.json.zip'
exps_filename = 'expenses.zip'

r = requests.get(exps_URL)

with open(downloads / exps_filename, mode='wb') as f:
    f.write(r.content)

with ZipFile(downloads /exps_filename, 'r') as zip_f:
    zip_f.extractall(downloads)

## Análise de Gastos

In [9]:
with open(downloads / 'Ano-2023.json', encoding='utf-8') as j:
    parsed = json.loads(j.read())

In [10]:
df = pd.json_normalize(parsed['dados'])

df = (
    df[
        # (df['descricao'] == 'DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.') &
        (df['legislatura'] == 2023)
    ]
    .drop(
        columns=[
            'cpf', 'numeroCarteiraParlamentar', 'legislatura',
            'siglaPartido', 'numeroSubCota', 'codigoLegislatura',
            'numeroEspecificacaoSubCota', 'passageiro',
            'trecho', 'parcela', 'datPagamentoRestituicao',
            'ressarcimento', 'descricaoEspecificacao',
            'restituicao', 'siglaUF', 'tipoDocumento'
            ]
        )
)

df = df.reset_index(drop=True)

df['cnpj_basico'] = df['cnpjCPF'].str.replace('.', '', regex=False).apply(lambda x: x[:8])

df['datetime'] = (
        (
            df['ano'].astype(str) +
            '-' +
            df['mes'].astype(str).str.zfill(2)
        ).apply(lambda x: pd.to_datetime(x, format='%Y-%m'))
    )

df = df[df['datetime'] < (datetime.today() - timedelta(days=60))]

In [12]:
(
    df[df['fornecedor'] == 'GOOGLE BRASIL INTERNET LTDA']
).to_csv(resultados / 'declaracoes-de-gastos-com-google.csv', index=False)

# Anúncios no Facebook

Não conseguimos encontrar uma maneira de buscar anúncios feitos na biblioteca de anúncios do Facebook a partir do nome ou de IDs de contas na plataforma.

A saída encontrada foi a de visitar cada página, raspar os anúncios feitos pelos perfis e posteriormente raspar as informações dos anúncios.

In [ ]:
driver = (
    webdriver
    .Firefox(service=FirefoxService(
        GeckoDriverManager()
        .install()
        )
    )
)

In [14]:
# IDs das páginas de pessoas políticas no Facebook. Coletadas manualmente.

pages_id = [
    '173698339802515', '865232070270799', '489383607761535',
    '1731337420462273', '1526870777612230', '1947356288874816',
    '1321564147955463', '107356795270257', '152534811624066',
    '104157151228447', '1638752696387328', '691899611250247',
    '1642363142681223', '286531148619744', '361989510562789',
    '345829265488872', '143803862450980', '391477497562861',
    '1507476039474128', '224968724875918', '112434351352176',
    '239756803177345', '368704519884205', '103015684871449',
    '529821053761365', '124307224406046', '924601577651630',
    '1527572800822387', '266360393867344', '166001994065417',
    '396626607095420', '100217516152889', '529741837114868',
    '216474455105209', '334810013335023', '1395013197441419',
    '358592104714583', '411754008869486', '272425403490751',
    '113901773672666', '221371211998801', '206365432837539',
    '611112112239401', '167306944111245', '781653925252644',
    '1433103303568739', '1533303356759820', '815039985305536',
    '410655005626777', '1526286600948051', '931215210253332',
    '420265928063107', '201991609984528', '179856875529257',
    '552636458116557', '393428077515241', '1082805205172897',
    '103860934882269', '514558765355005', '225167147655939',
    '201044363357878', '409540602424777', '220778897969256',
    '249383531739005', '695358380551267', '133602516654912',
    '111949165566730', '332870466782583', '217423271714827',
    '212423668965384', '122923285036030', '458048987893303',
    '266651300126832', '721618744570005', '237597346269787',
    '127139487368866', '580813708692882', '102088129018202',
    '1035223423186579', '124002814371381', '286648849567',
    '154086204688099', '193185607843864', '223832637675564',
    '191512767645241', '428795217537762', '547530398646435',
    '182235818872744', '100261572206297', '650190518395498',
    '1054901897864467', '104530757861973', '123097244540886',
    '842245655790289', '102753402027906', '699693400101917',
    '205362089591468', '345425238826528', '659203737425877',
    '117903068817531', '2042050096056582', '646292118914142',
    '106376318992146', '140347022821700', '1441794469233984',
    '1695544247372940', '775240779164982', '1191229370929329',
    '272809098799', '851328134910897', '314213892042373',
    '1450940791842660', '156499414480143', '136344939876101',
    '660410357307599', '177115472342855', '1164910073641031',
    '533495260042236', '102343683293212', '301814029885718',
    '187682584611376', '564150546986457', '158198544337175',
    '517712008679531', '329948620416635', '471144916255739',
    '824390270907057', '765667336867771', '301106093563904',
    '278887206056242', '575472512602115', '390501545079417',
    '191591598062172', '102363272346398', '223383744823290',
    '114407660269758', '208610802492299', '394517827279852',
    '222639301175182', '212265828905225', '250817031788413',
    '508569109204694', '680238528731583', '552483638152177',
    '104486751876738', '364253843645006', '835813059780644',
    '217451785552263', '1931989763744839', '251489518948731',
    '155201831260401', '328393683976480', '110198205136477',
    '496443053731900', '227403640631475', '130558143745099',
    '520798404655508', '169743843175117', '1378617549025463',
    '405401689514244', '1401978510018003', '782277955151459',
    '518031775303071', '833053646745836', '678761768950811',
    '1619570641615465', '1414776178738297', '620314628044090',
    '108143861587898', '176976939109000', '273642106068512',
    '1433929133492494', '200263040068985', '238761896484083',
    '433519783479725', '462223517218816', '158894974200303',
    '420875917999748', '861928933853138', '397405070321390',
    '340186179736898', '555803774486872', '245784392128312',
    '632161500194120', '263563620333111', '953514998164299',
    '687008454694990', '405696379766191', '486914985073732',
    '1385505868350890', '578898368860170', '206019209573311',
    '142099522556207', '157982450937286', '174471285940778',
    '1142941029175690', '1501236886772782', '401673676620654',
    '656965247684115', '225122704193171', '424808937641023',
    '174217422747805', '274527822566791', '1720125068073321',
    '729392980520143', '1463458067229810', '235356953965179',
    '1460558354166610', '194032184000941', '364042537032048',
    '457224910969875', '801126203398977', '291482090931929']


In [15]:
start_date = datetime(2023, 1, 1)
day_list = rrule(freq=DAILY, dtstart=start_date, until=datetime.now())

days = []
for start_date in day_list:
    days.append((start_date, start_date + timedelta(days=1)))

In [ ]:
saved_files = [file.name for file in ad_library_main_page.glob('*.html')]

for page_id in pages_id:
    for day_n, day in enumerate(days):
        start_date = day[0].strftime('%Y-%m-%d')
        end_date = day[1].strftime('%Y-%m-%d')

        url = f'https://www.facebook.com/ads/library/?active_status=all&ad_type=all&country=ALL&view_all_page_id={page_id}&sort_data[direction]=desc&sort_data[mode]=relevancy_monthly_grouped&start_date[min]={start_date}&start_date[max]={end_date}&search_type=page&media_type=all'
        filename = f'{page_id}_{day_n}.html'

        if filename not in saved_files:
            driver.get(url)
            for i in range(20):
                sleep(0.5)
                ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
                ActionChains(driver).send_keys(Keys.END).perform()
            with open(ad_library_main_page / filename, mode='w', encoding='utf-8') as f:
                f.write(driver.page_source)


In [17]:
def get_all_ads_ids(soup):
    ads_id = []

    for i in soup.find_all('div'):
        if 'Identificação' in i.text:
            if len(i.text.split()) == 2:
                ad_id = i.text.split()[-1]
                ads_id.append(ad_id)
    
    ads_id = list(set(ads_id))
    
    return ads_id

In [ ]:
ads_collected = [file.name for file in single_ads_pages.glob('*.html')]

for file in ad_library_main_page.glob('*.html'):
    ads_in_file = []

    with open(file) as ad_catalogue:
        soup = bs(ad_catalogue.read(), 'html.parser')
        ads_in_file.extend(get_all_ads_ids(soup))
    
    for ad_id in ads_in_file:
        filename = f'{ad_id}.html'
        if filename not in ads_collected:
            sleep(3)
            driver.get(f'https://www.facebook.com/ads/library/?id={ad_id}')
            sleep(5)
            soup = bs(driver.page_source, 'html.parser')
            modal = soup.find('div', {'aria-label': 'Modal for the deep link ad'})
            with open(single_ad_pages / filename, mode='w', encoding='utf-8') as f:
                f.write(modal.contents[0].__str__())
                
                # Atualiza os ads coletados somente quando um novo é salvo.
                ads_collected = [file.name for file in single_ad_pages.glob('*.html')]
    
    os.rename(file, ad_library_main_page / f'{file.stem}.old')

In [ ]:
def parse_html(soup):
    def get_ad_owner(soup):
        try:
            ad_owner = (
                soup
                .find('img')
                ['alt']
            )
        except TypeError:
            ad_owner = (
                soup
                .find('a')
                .text
            )
        return ad_owner
    
    def get_ad_id(soup):
        ad_string = (
            re.findall(
                string=soup.text,
                pattern='Identificação: \d{6,}'
                )
                )[0]
        ad_id = ad_string.split(':')[-1].strip()
        return ad_id

    def get_impressions_count(soup):
        for e in reversed(soup.find_all('div')):
            if 'Impressões' in e.text:
                impressions = unicodedata.normalize('NFKD', e.text)
                break
        if '<1 mil' in impressions:
            lower_impressions, upper_impressions = (0, 1000)
        elif '>1 mi' in impressions:
            lower_impressions, upper_impressions = (1_000_000, None)
        else:
            impressions = (
                impressions
                .split(':')[-1]
                .strip()
                .replace(' mil', '000')
                .replace(' mi', '000000')
                )
            lower_impressions, upper_impressions = impressions.split(' a ')

        return lower_impressions, upper_impressions

    def get_spent_value(soup):
        for e in reversed(soup.find_all('div')):
            if 'Valor gasto' in e.text:
                spent_value = unicodedata.normalize('NFKD', e.text)
                break
        
        spent_value = (
            spent_value
            .split(':')[-1]
            .strip()
            .replace('R$', '')
            )
        lower_spent_value, upper_spent_value = spent_value.split(' a ')

        return int(lower_spent_value), int(upper_spent_value)

    def get_spent_value(soup):
        for e in reversed(soup.find_all('div')):
            if 'Valor gasto' in e.text:
                spent_value = unicodedata.normalize('NFKD', e.text)
                break
        if '<R$100' in spent_value:
            lower_spent_value, upper_spent_value = 0, 100

        else:
            spent_value = (
                spent_value
                .split(':')[-1]
                .strip()
                .replace('R$', '')
                .replace(',5 mil', '500')
                .replace(' mil', '000')
                )
            lower_spent_value, upper_spent_value = spent_value.split(' a ')

        return int(lower_spent_value), int(upper_spent_value)
    
    def get_publication_range(soup):
        def format_date(date):
            replace_map = {
                'jan': '01', 'fev': '02', 'mar': '03',
                'abr': '04', 'mai': '05', 'jun': '06',
                'jul': '07', 'ago': '08', 'set': '09',
                'out': '10',  'nov': '11', 'dez': '12'
                }
            day, month, year = date.lower().split('/')
            day = day.zfill(2)
            month = replace_map[month]
            date = '/'.join([day, month, year])
            return date

        months = [
            ' jan ', ' fev ', ' mar ', ' abr ',
            ' mai ', ' jun ', ' jul ', ' ago ', ' set ',
            ' out ', ' nov ', ' dez '
            ]
        for e in reversed(soup.find_all('div')):
            if any (month in e.text for month in months):
                if re.search(string=e.text, pattern='\d{1,2} de \w{3} de \d{4}'):
                    publication_range = (
                        e.text
                        .replace(' de ', '/')
                        )
                    break
        
        if 'Veiculação iniciada' in publication_range:
            start_date = publication_range.replace('Veiculação iniciada em ', '')
            start_date = format_date(start_date)
            end_date = None
        
        else:
            start_date, end_date = publication_range.split(' a ')
            start_date = format_date(start_date)
            end_date = format_date(end_date)
        
        return start_date, end_date

    ad_id = get_ad_id(soup)
    try:
        ad_owner = get_ad_owner(soup)
        lower_impressions, upper_impressions = get_impressions_count(soup)
        lower_spent, upper_spent = get_spent_value(soup)
        available_date_min, available_date_max = get_publication_range(soup)
    except UnboundLocalError:
        print(ad_id)
        ad_owner = None
        lower_impressions = None
        upper_impressions = None
        lower_spent = None
        upper_spent = None
        available_date_min = None
        available_date_max = None

    info = {
        'ad_URL': 'https://www.facebook.com/ads/library/?id=' + ad_id,
        'ad_id': ad_id,
        'ad_owner': ad_owner,
        'lower_impressions_range': lower_impressions,
        'upper_impressions_range': upper_impressions,
        'lower_spent_range_BRL': lower_spent,
        'upper_spent_range_BRL': upper_spent,
        'available_date_min': available_date_min,
        'available_date_max': available_date_max
    }
    
    return info

In [ ]:
infos = []
for file in single_ads_pages.glob('*.html'):
    with open(file, mode='r', encoding='utf8') as f:
        html_content = f.read()
        soup  = bs(html_content, 'html.parser')
        parsed = parse_html(soup)
        infos.append(parsed)

In [ ]:
df_facebook = pd.DataFrame(infos)

In [ ]:
df_facebook.to_csv(resultados / 'ads-no-facebook-de-congressistas-2023.csv', index=False)